In [1]:
import torchvision
from torchvision.transforms import ToTensor

from PIL import Image
import os
import glob
import random
random.seed(4)

import pandas as pd
import numpy as np
import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
import torchvision.models as models
import torch.optim as optim

import json
import pandas as pd
from PIL import Image
import requests
from io import BytesIO


## Data

In [2]:
d = {'all':[]}
with open("/home/yagor/Рабочий стол/yandex generation/load data/pictures_market.txt") as infile:
    for line in infile:
        result = json.loads(line)
        d['all'].append(result)
df = pd.DataFrame(d['all'])

In [3]:
df['verdict'].value_counts()

clean_photo_good_background     467031
good_infographics               300772
other_infographics              219703
bad_infographics                 50719
clean_photo_other_background     50496
clean_photo_bad_background       28440
clean_photo_image_background     14671
Name: verdict, dtype: int64

### info есть/нет

In [ ]:
infographic = {1: [ 'good_infographics', 'bad_infographics', 'other_infographics'], 0: ['clean_photo_image_background', 'clean_photo_good_background', 'clean_photo_bad_background']}

In [ ]:
info_df = df.loc[df['verdict'].isin(infographic[1]+infographic[0])]

In [ ]:
PERCENT_FRAC = 0.2

In [ ]:
sample_df = info_df.groupby('verdict', as_index=False).apply(lambda x: x.sample(frac=PERCENT_FRAC, random_state=11)).reset_index(drop=True)

In [ ]:
sample_df["verdict"] = np.where(sample_df["verdict"].isin(infographic[0]), 0, 1)

### bad/good info

In [5]:
infographic = {1: [ 'good_infographics'], 0: ['bad_infographics']}

In [6]:
info_df = df.loc[df['verdict'].isin(infographic[1]+infographic[0])]

In [7]:
info_df

,market_sku_id,pic_url,verdict
4,429050296,//avatars.mds.yandex.net/get-marketpic/8786714...,good_infographics
6,299703908,//avatars.mds.yandex.net/get-marketpic/1878151...,good_infographics
8,186523793,//avatars.mds.yandex.net/get-marketpic/9247093...,good_infographics
10,101872355836,//avatars.mds.yandex.net/get-marketpic/7472696...,good_infographics
14,101814268773,//avatars.mds.yandex.net/get-marketpic/5415535...,good_infographics
...,...,...,...
1131819,101781815789,//avatars.mds.yandex.net/get-marketpic/8259349...,good_infographics
1131821,101920408233,//avatars.mds.yandex.net/get-marketpic/8604745...,good_infographics
1131826,431075969,//avatars.mds.yandex.net/get-marketpic/6018159...,bad_infographics
1131829,101815744136,//avatars.mds.yandex.net/get-marketpic/7067214...,good_infographics


In [8]:
di = {'bad_infographics':0, 'good_infographics': 1}

In [9]:
info_df['verdict'].value_counts()

good_infographics    300772
bad_infographics      50719
Name: verdict, dtype: int64

In [10]:
info_df = info_df.replace({'verdict': di})

In [11]:
info_df['verdict'].value_counts()

1    300772
0     50719
Name: verdict, dtype: int64

In [12]:
index_drop = info_df[info_df['verdict']==1].sample(frac=1).index

In [13]:
real_index_drop = index_drop[80000:]
real_index_drop

Int64Index([ 48179, 553694, 934342, 611737,  55915,  65486, 996666, 400951,
            673298, 131684,
            ...
            501124, 378866, 746490, 838348, 754824,  44245,  85937, 331886,
            276165, 330593],
           dtype='int64', length=220772)

In [14]:
sample_df = info_df.drop(real_index_drop)
sample_df['verdict'].value_counts()

1    80000
0    50719
Name: verdict, dtype: int64

### bad/good/image back

In [83]:
di = {'clean_photo_good_background':0, 'clean_photo_image_background': 1, 'clean_photo_bad_background': 2}

In [84]:
di.keys()

dict_keys(['clean_photo_good_background', 'clean_photo_image_background', 'clean_photo_bad_background'])

In [85]:
info_df = df.loc[df['verdict'].isin(di.keys())]

In [86]:
info_df = info_df.replace({'verdict': di})

In [89]:
info_df

,market_sku_id,pic_url,verdict
0,1402035435,//avatars.mds.yandex.net/get-marketpic/7044394...,0
1,1402035435,//avatars.mds.yandex.net/get-marketpic/5620596...,0
3,729239006,//avatars.mds.yandex.net/get-marketpic/1897710...,0
5,429050296,//avatars.mds.yandex.net/get-marketpic/1857844...,0
12,101858930853,//avatars.mds.yandex.net/get-marketpic/1776500...,0
...,...,...,...
1131824,677173499,//avatars.mds.yandex.net/get-marketpic/5115066...,0
1131825,101121991197,//avatars.mds.yandex.net/get-marketpic/1662968...,0
1131827,100818966350,//avatars.mds.yandex.net/get-marketpic/1697981...,0
1131828,100972165930,//avatars.mds.yandex.net/get-marketpic/1582458...,0


In [90]:
info_df['verdict'].value_counts()

0    467031
2     28440
1     14671
Name: verdict, dtype: int64

In [93]:
index_drop = info_df[info_df['verdict']==0].sample(frac=1).index

In [94]:
real_index_drop = index_drop[50000:]
real_index_drop

Int64Index([ 922191,  210130,  695502,  663588,  894081,  221172,  300237,
             213146, 1015472,  991262,
            ...
             271206,  117693,   36148,   74929, 1094829,  953533, 1008743,
             458006, 1045584,  391561],
           dtype='int64', length=417031)

In [95]:
sample_df = info_df.drop(real_index_drop)
sample_df['verdict'].value_counts()

0    50000
2    28440
1    14671
Name: verdict, dtype: int64

## Create datasets

In [15]:
from sklearn.model_selection import train_test_split

data_train, data_val = train_test_split(sample_df, test_size=0.2, random_state=42)
data_train.index = range(len(data_train))
data_val.index = range(len(data_val))

print("Train size = ", len(data_train))
print("Validation size = ", len(data_val))

Train size =  104575
Validation size =  26144


In [16]:
class MyDataset(torch.utils.data.IterableDataset):
    def __init__(self, df, mode = 'train', transform=None):
        self.data = df
        self.transform = transform
        self.mode = mode
        self.indexs = df.index.tolist()
        self.bad_indexs = {}
         
    def __iter__(self):
         # yield only valid data, skip Nones
        for idx in self.indexs:
            try:
                if idx in self.bad_indexs:
                    continue
                else:
                    url = self.data.loc[idx, 'pic_url']
                    response = requests.get('https:'+ url)
                    im = Image.open(BytesIO(response.content)).convert('RGB')
                    if self.mode=='train':
                        label = self.data.loc[idx, 'verdict']
                        label_tensor =  torch.as_tensor(label, dtype=torch.long)
                
                    if self.transform:
                        im = self.transform(im)

                    if self.mode == 'test':
                        # For saving the predictions, the file name is required
                        yield {'im' : im}
                    else:
                        yield {'im' : im, 'labels' : label_tensor}
            except:
                self.bad_indexs[url] = True
        print(str(len(self.bad_indexs)))

    def __len__(self):
        return len(self.data)

    def shuffle(self):
        # call this to shuffle the dirs between epochs
        #self.dirs = [self.dirs[idx] for idx in torch.utils.data.RandomSample(range(len(self.indexs)))]
        sample_df = info_df.groupby('verdict', as_index=False).apply(lambda x: x.sample(frac=PERCENT_FRAC, random_state=random.randint(0, 300))).reset_index(drop=True)
        sample_df["verdict"] = np.where(sample_df["verdict"].isin(infographic[0]), 0, 1)
        data_train, _ = train_test_split(sample_df, test_size=0.2, random_state=random.randint(0, 125))
        self.data = data_train

In [17]:
# Channel wise mean and standard deviation for normalizing according to ImageNet Statistics
means =  [0.485, 0.456, 0.406]
stds  =  [0.229, 0.224, 0.225]

size = 70
# Transforms to be applied to Train-Test-Validation
train_transforms      =  transforms.Compose([
                         #transforms.RandomRotation(30),
                         transforms.Resize(size),
                         transforms.CenterCrop(size),
                         #transforms.RandomHorizontalFlip(p=0.5),
                         transforms.ToTensor(),
                         transforms.Normalize(means, stds)])

test_valid_transforms =  transforms.Compose([
                         transforms.Resize(size),
                         transforms.CenterCrop(size),
                         transforms.ToTensor(),
                         transforms.Normalize(means, stds)])

In [18]:
train_dataset = MyDataset(data_train, transform=train_transforms)
valid_dataset = MyDataset(data_val, transform=test_valid_transforms)

In [19]:
BATCH_SIZE = 200
N_CORES = 4

In [20]:
len(train_dataset.data.loc[:, 'verdict'].values)

104575

In [21]:
type(train_dataset)

__main__.MyDataset

In [22]:
train_loader =  torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, num_workers=N_CORES)
val_loader   =  torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, num_workers=N_CORES)

data_loaders =  {'train' : train_loader, 'valid': val_loader}

## model Dino v2

In [23]:

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import random

In [24]:
SEED = 42
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
# Set seed
seed_everything(SEED)

In [25]:
dinov2_vits14 = torch.hub.load('facebookresearch/dinov2', 'dinov2_vits14')

Using cache found in /home/yagor/.cache/torch/hub/facebookresearch_dinov2_main


In [26]:

class DinoVisionTransformerClassifier(nn.Module):
    def __init__(self):
        super(DinoVisionTransformerClassifier, self).__init__()
        self.transformer = dinov2_vits14
        self.classifier = nn.Sequential(
            nn.Linear(384, 256),
            nn.ReLU(),
            nn.Linear(256, 2)
        )
    
    def forward(self, x):
        x = self.transformer(x)
        x = self.transformer.norm(x)
        x = self.classifier(x)
        return x

In [27]:

import torch.optim as optim

model = DinoVisionTransformerClassifier()

In [29]:
criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)
optimizer = optim.Adam(model.parameters(), lr=0.000001)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [30]:
model = model.to(device)

In [139]:
for epoch in range(1):  # loop over the dataset multiple times

    running_loss = 0.0
    max_iter = len(train_dataset)//BATCH_SIZE
    min_loss = np.inf
    train_data_loader = tqdm.tqdm(data_loaders['train'])
    for i, data in enumerate(train_data_loader):
        # get the inputs; data is a list of [inputs, labels]
        if i>=max_iter:
            break

        inputs, labels = data['im'].to(device), data['labels'].to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # torch.save(model, 'pretrain_model_transformer_info_not_info.pt')

        if loss.item()<min_loss:
            min_loss = loss.item()
            torch.save(model, 'pretrain_model_transformer_good_bad_info_2class.pt')


        # print statistics
        running_loss += loss.item()
        if i % 50 == 49:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 50:.3f}')
            running_loss = 0.0

print('Finished Training')

 10%|▉         | 51/523 [09:21<1:05:44,  8.36s/it]

[1,    50] loss: 0.652


 11%|█▏        | 60/523 [10:52<50:06,  6.49s/it]  /home/yagor/anaconda3/envs/detect/lib/python3.10/site-packages/PIL/Image.py:979: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/home/yagor/anaconda3/envs/detect/lib/python3.10/site-packages/PIL/Image.py:979: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/home/yagor/anaconda3/envs/detect/lib/python3.10/site-packages/PIL/Image.py:979: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/home/yagor/anaconda3/envs/detect/lib/python3.10/site-packages/PIL/Image.py:979: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
 19%|█▉        | 100/523 [17:44<40:22,  5.73s/it] 

[1,   100] loss: 0.590


 29%|██▉       | 151/523 [27:05<54:46,  8.83s/it]  

...

[1,   450] loss: 0.511


 96%|█████████▌| 500/523 [1:27:44<02:05,  5.45s/it]

[1,   500] loss: 0.534


100%|█████████▉| 522/523 [1:32:11<00:10, 10.60s/it]

Finished Training


In [141]:
correct = 0
total = 0
# out = []
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    valid_data_loader = tqdm.tqdm(data_loaders['valid'])
    max_iter_val = len(valid_dataset)//BATCH_SIZE
    model.eval()

    for i, data in enumerate(valid_data_loader):
        if i>=max_iter_val:
            break
        images, labels = data['im'].to(device), data['labels']
        # calculate outputs by running images through the network
        outputs = model(images.to(device))
        # the class with the highest energy is what we choose as prediction
        outmax, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted.to("cpu") == labels).sum().item()
        # out.append(outmax.to("cpu"))

print(f'Accuracy of the network on the test images: {100 * correct // total} %')

 29%|██▉       | 38/131 [06:28<14:34,  9.41s/it] /home/yagor/anaconda3/envs/detect/lib/python3.10/site-packages/PIL/Image.py:979: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/home/yagor/anaconda3/envs/detect/lib/python3.10/site-packages/PIL/Image.py:979: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/home/yagor/anaconda3/envs/detect/lib/python3.10/site-packages/PIL/Image.py:979: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
/home/yagor/anaconda3/envs/detect/lib/python3.10/site-packages/PIL/Image.py:979: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
 63%|██████▎   | 83/131 [14:22<07:06,  8.89s/it]

In [35]:
100 * correct / total

86.27551020408163

## Gradio

In [32]:
# Channel wise mean and standard deviation for normalizing according to ImageNet Statistics
means =  [0.485, 0.456, 0.406]
stds  =  [0.229, 0.224, 0.225]

size = 70
# Transforms to be applied to Train-Test-Validation
train_transforms      =  transforms.Compose([
                         #transforms.RandomRotation(30),
                         transforms.Resize(size),
                         transforms.CenterCrop(size),
                         #transforms.RandomHorizontalFlip(p=0.5),
                         transforms.ToTensor(),
                         transforms.Normalize(means, stds)])

test_valid_transforms =  transforms.Compose([
                         transforms.Resize(size),
                         transforms.CenterCrop(size),
                         transforms.ToTensor(),
                         transforms.Normalize(means, stds)])

In [33]:
import gradio as gr
def create_model(MODEL_PATH, device=None):
    model = torch.load(MODEL_PATH)
    if not device:
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()
    return model
classes = ['good_infographics', 'bad_infographics', 'other_infographics', 'clean_photo_image_background', 'clean_photo_good_background', 'clean_photo_bad_background']
model_info = create_model('/home/yagor/Рабочий стол/yandex generation/load data/pretrain_model_transformer_info_not_info_2claas.pt')
model_info_bad_good = create_model('/home/yagor/Рабочий стол/yandex generation/load data/pretrain_model_transformer_good_bad_info_2class.pt')
model_clean_image_bad_back = create_model('/home/yagor/Рабочий стол/yandex generation/load data/pretrain_model_transformer_good_bad_image_back.pt')
models = {'Есть/нет инфографика': [model_info, {0:'Инфографики нет', 1:'Инфографика есть'}],
 'Хорошая/плохая инфографика': [model_info_bad_good, {0:'bad_infographics', 1:'good_infographics'}],
 'Какой фон': [model_clean_image_bad_back, {0:'clean_photo_good_background', 1:'clean_photo_image_background', 2:'clean_photo_bad_background'}]}

In [61]:


def predict(model, inp, device=None):
    with torch.no_grad():
        if not device:
            device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        outputs = model(inp.to(device))
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        predicted = predicted.to('cpu')
    return torch.nn.functional.softmax(outputs[0], dim=0), predicted
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


def image_classification(inp, models_choice):
    model, classes = models[models_choice]
    inp = test_valid_transforms(inp).unsqueeze(0)
    predictions, predicted = predict(model, inp)
    confidences = {classes[i]: float(predictions[i]) for i in range(len(predictions))}
    return confidences


In [96]:
import gradio as gr
gr.Interface(fn=image_classification,
             inputs=[
            gr.Image(type="pil"), gr.Dropdown(['Есть/нет инфографика', 'Хорошая/плохая инфографика', 'Какой фон'], label="Выберите модель", value='Есть/нет инфографика'), ],
             outputs=gr.Label(num_top_classes=2),
             examples=[["/home/yagor/Рабочий стол/yandex generation/load data/8794f117-98d8-494e-90d7-cbc2cedfa167.jpeg"
             ]]
             ).launch(share=True)

Running on local URL:  http://127.0.0.1:7878

To create a public link, set `share=True` in `launch()`.
